# **0. 참고자료**
## 0-1. 도서  
- 비제이퍼블릭 | 한주현 저-바이오파이썬으로 만나는 생물정보학  

## 0-2. 논문, 학술지

## 0-3. 웹 사이트
- wikipedia | [위키피디아 링크](https://ko.wikipedia.org/wiki/)

## 0-4. 데이터셋 출처


# **1.Entrez**
- NCBI의 방대한 데이터 검색 시스템으로 PubMed, Genbank, GEO 등 49개 DB 접근을 제공  
- [[NCBI Entrez 검색 페이지]](https://www.ncbi.nlm.nih.gov/search) 
- Entrez를 안정적인 시스템 운영과 사용자들의 편리한 이용을 위한 가이드라인을 제시하였다.  
    1) 100개 이상의 질의가 있다면 주말에는 미국 시간 기준 바쁜 시간 이후에 사용해야한다.
    2) 초당 3개 이상의 질의를 날리면 안된다.
    3) 과도한 사용으로 차단사실을 알림받기 위해 유효한 이메일을 입력해야한다.

**(!) 위의 가이드 라인은 biopython을 사용할 떄 신경쓰지 않아도 된다.**

## 1-1. biopython.efetch()로 poliovirus 데이터 읽기
**(!) Poliovirus는 소아마비의 병원체로서 피코르나바이러스과 엔테러바이러스 C종의 혈쳥형이다.**
- efetch()는 Entrez의 모든 레코드를 받는 유틸리티이다.
- NCBI의 DB는 대부분 여러가지 포맷으로 결과를 제공할 수 있으므로 rettype과 retmode에서 결과 포맷을 지정해야한다.
- Entrez 시스템을 이용해야 하므로, 네트워크가 연결되어 있어야한다.

In [1]:
from Bio import Entrez

## 유효한 메일주소를 입력해준다. 
Entrez.email = "kimhippo401@gmail.com"

## Entrez에서 데이터를 받아온다.
handle = Entrez.efetch(db = "nucleotide", id="NC_002058.3", rettype="gb", retmode="text")
print(handle.read())

LOCUS       NC_002058               7440 bp ss-RNA     linear   VRL 13-AUG-2018
DEFINITION  Poliovirus, complete genome.
ACCESSION   NC_002058 NC_001428 NC_014336
VERSION     NC_002058.3
DBLINK      BioProject: PRJNA485481
KEYWORDS    RefSeq; coat protein; complementary DNA; genome; polyprotein.
SOURCE      Enterovirus C
  ORGANISM  Enterovirus C
            Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes;
            Picornavirales; Picornaviridae; Ensavirinae; Enterovirus.
REFERENCE   1  (bases 1 to 742)
  AUTHORS   Kafasla,P., Morgner,N., Robinson,C.V. and Jackson,R.J.
  TITLE     Polypyrimidine tract-binding protein stimulates the poliovirus IRES
            by modulating eIF4G binding
  JOURNAL   EMBO J. 29 (21), 3710-3722 (2010)
   PUBMED   20859255
REFERENCE   2  (bases 1 to 742)
  AUTHORS   Vogt,D.A. and Andino,R.
  TITLE     An RNA element at the 5'-end of the poliovirus genome functions as
            a general promoter for RNA synthesis
  JOURNAL   PLoS Path

## 1-2. EntrezXML로 EFetch 데이터 파싱하기
- Biopython에서 Entrez의 xml 형식의 질의결과를 파싱해주는 2가지 메소드를 제공한다.
1) Entrez.read()  
   → 리스트 형태로 반환한다.  
   → 메모리 부담이 크다  
   → xml을 읽은 모든 레코드 결과를 한 번에 메모리에 올려 사용한다.
   
2) Entrez.parse()  
   → 제너레이터 형태로 반환한다.  
   → 메모리 부담이 적다  
   → 레코드 하나하나를 그때마다 읽어 파싱한 결과를 반환한다.

In [2]:
%%time
records = Entrez.efetch(db = "nucleotide", id="NC_002058.3", rettype="gb", retmode="xml")
records = Entrez.read(records)

for record in records:
    print(f'locus : {record["GBSeq_locus"]}')
    print(f'definition : {record["GBSeq_definition"]}')
    print(f'Strandedness : {record["GBSeq_strandedness"]}')
    print(f'moltype : {record["GBSeq_moltype"]}')
    print(f'length : {record["GBSeq_length"]} bp')
    print(f'journals : {record["GBSeq_references"]}')
print('\n')

locus : NC_002058
definition : Poliovirus, complete genome
Strandedness : single
moltype : RNA
length : 7440 bp
journals : [{'GBReference_reference': '1', 'GBReference_position': '1..742', 'GBReference_authors': ['Kafasla,P.', 'Morgner,N.', 'Robinson,C.V.', 'Jackson,R.J.'], 'GBReference_title': 'Polypyrimidine tract-binding protein stimulates the poliovirus IRES by modulating eIF4G binding', 'GBReference_journal': 'EMBO J. 29 (21), 3710-3722 (2010)', 'GBReference_xref': [{'GBXref_dbname': 'doi', 'GBXref_id': '10.1038/emboj.2010.231'}], 'GBReference_pubmed': '20859255'}, {'GBReference_reference': '2', 'GBReference_position': '1..742', 'GBReference_authors': ['Vogt,D.A.', 'Andino,R.'], 'GBReference_title': "An RNA element at the 5'-end of the poliovirus genome functions as a general promoter for RNA synthesis", 'GBReference_journal': 'PLoS Pathog. 6 (6), E1000936 (2010)', 'GBReference_xref': [{'GBXref_dbname': 'doi', 'GBXref_id': '10.1371/journal.ppat.1000936'}], 'GBReference_pubmed': '2

In [3]:
%%time
records = Entrez.efetch(db = "nucleotide", id="NC_002058.3", rettype="gb", retmode="xml")
records = Entrez.parse(records)

for record in records:
    print(f'locus : {record["GBSeq_locus"]}')
    print(f'definition : {record["GBSeq_definition"]}')
    print(f' : {record["GBSeq_strandedness"]}')
    print(f'Strandedness moltype : {record["GBSeq_moltype"]}')
    print(f'length : {record["GBSeq_length"]} bp')
    print(f'journals : {record["GBSeq_references"]}')
print('\n')

locus : NC_002058
definition : Poliovirus, complete genome
 : single
Strandedness moltype : RNA
length : 7440 bp
journals : [{'GBReference_reference': '1', 'GBReference_position': '1..742', 'GBReference_authors': ['Kafasla,P.', 'Morgner,N.', 'Robinson,C.V.', 'Jackson,R.J.'], 'GBReference_title': 'Polypyrimidine tract-binding protein stimulates the poliovirus IRES by modulating eIF4G binding', 'GBReference_journal': 'EMBO J. 29 (21), 3710-3722 (2010)', 'GBReference_xref': [{'GBXref_dbname': 'doi', 'GBXref_id': '10.1038/emboj.2010.231'}], 'GBReference_pubmed': '20859255'}, {'GBReference_reference': '2', 'GBReference_position': '1..742', 'GBReference_authors': ['Vogt,D.A.', 'Andino,R.'], 'GBReference_title': "An RNA element at the 5'-end of the poliovirus genome functions as a general promoter for RNA synthesis", 'GBReference_journal': 'PLoS Pathog. 6 (6), E1000936 (2010)', 'GBReference_xref': [{'GBXref_dbname': 'doi', 'GBXref_id': '10.1371/journal.ppat.1000936'}], 'GBReference_pubmed': '

**(!) 위 테스트를 통해, Entrez.parse()가 Entrez.read()보다 2배 이상 빠름을 보였다.**

## 1-3. EInfo를 통해 Entrez의 데이터베이스 목록 확인하기
- 바이오파이썬에서는 Entrez.einfo()를 통해 Entrez에서 제공하는 데이터 베이스를 확인한다.
- Entrez.read() 메소드를 통해 Entrez.einfo()의 결과물을 딕셔너리 형으로 변환할 수 있다

In [4]:
databases = Entrez.einfo()
databases = Entrez.read(databases)

for idx, database in enumerate(databases['DbList'], 1):
    print(f'[{idx}] - {database}')

[1] - pubmed
[2] - protein
[3] - nuccore
[4] - ipg
[5] - nucleotide
[6] - structure
[7] - genome
[8] - annotinfo
[9] - assembly
[10] - bioproject
[11] - biosample
[12] - blastdbinfo
[13] - books
[14] - cdd
[15] - clinvar
[16] - gap
[17] - gapplus
[18] - grasp
[19] - dbvar
[20] - gene
[21] - gds
[22] - geoprofiles
[23] - homologene
[24] - medgen
[25] - mesh
[26] - nlmcatalog
[27] - omim
[28] - orgtrack
[29] - pmc
[30] - popset
[31] - proteinclusters
[32] - pcassay
[33] - protfam
[34] - pccompound
[35] - pcsubstance
[36] - seqannot
[37] - snp
[38] - sra
[39] - taxonomy
[40] - biocollections
[41] - gtr
